In [1]:
import os
from pathlib import Path
path = Path(os.getcwd())
os.chdir(path.parent.absolute())

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import mt4_hst
%matplotlib inline

# pd.options.display.max_rows = 9999

In [ ]:
ticker = 'FPT'
df = mt4_hst.read_hst("../stock_datasets/" + ticker + "1440.hst")
df = df.sort_values('time')
df.sort_values(by='time', inplace=True)

In [2]:
TechnicalIndicator = ta.Strategy(
    name="Standard Technical Indicator in various research of automatic stock trading",
    description="SMA MACD RSI CCI ADX Bollinger",
    ta=[
        {"kind": "sma", "length": 10},
        {"kind": "rsi", "length": 14},
        {"kind": "cci", "length": 14},
        {"kind": "adx", "length": 14},
        {"kind": "bbands"},
        {"kind": "macd"},
    ]
)

In [51]:
tickers = "VNM FPT".split()
df = pd.DataFrame()
for ticker in tickers:
    _df = mt4_hst.read_hst("../stock_datasets/" + ticker + "1440.hst")
    _df = _df[_df['time'] >= '2018-01-01']
    _df.sort_values(by='time', inplace=True)
    _df['ticker'] = ticker
    _df.ta.strategy(TechnicalIndicator)
    df = pd.concat([df, _df])

df = df.pivot(columns=['ticker']).swaplevel(0, 1, axis=1).sort_index(axis=1)
df.dropna(inplace=True)
df.columns.set_names(['ticker', 'indicators'], inplace=True)
df.head()

ticker            FPT                                                       \
indicators     ADX_14 BBB_5_2.0  BBL_5_2.0  BBM_5_2.0 BBP_5_2.0  BBU_5_2.0   
3010        13.851602  2.640890  21.497491  21.785152  0.133929  22.072813   
3011        13.520983  3.009184  21.437136  21.764605  0.209802  22.092073   
3012        12.677717  3.009184  21.437136  21.764605  0.601961  22.092073   
3013        11.894684  2.250783  21.473957  21.718373  0.573559  21.962790   
3014        11.521861  1.926801  21.488789  21.697826  0.635156  21.906863   

ticker                                                      ...           VNM  \
indicators CCI_14_0.015     DMN_14     DMP_14 MACD_12_26_9  ... MACDh_12_26_9   
3010          40.520133  27.397944  22.770901    -0.075523  ...      0.398797   
3011          24.071003  25.912920  21.536672    -0.078584  ...      0.247801   
3012          69.613235  23.699520  22.900215    -0.059599  ...      0.157328   
3013          61.669685  22.657427  21.893267    -0.050192  ...      0.230796   
3014          20.613365  24.305790  21.263939    -0.042249  ...      0.395270   

ticker                                                                   \
indicators MACDs_12_26_9     RSI_14      SMA_10       close        high   
3010           -1.944078  45.936886  114.351438  115.500671  116.655678   
3011           -1.882128  41.463615  114.103112  113.190659  115.269669   
3012           -1.842796  41.463615  114.039587  113.190659  114.923164   
3013           -1.785097  46.452741  114.518915  115.096420  115.211914   
3014           -1.686280  50.703324  115.119518  116.886681  118.388184   

ticker                                                   
indicators         low        open       time    volume  
3010        114.345665  116.655678 2018-02-27  457260.0  
3011        113.190659  114.923164 2018-02-28  886780.0  
3012        113.190659  113.190659 2018-03-01  732890.0  
3013        112.613152  113.190659 2018-03-02  412780.0  
3014        114.634415  115.385170 2018-03-05  712070.0  

[5 rows x 40 columns]

In [60]:
indicators = "close ADX_14 SMA_10 RSI_14 CCI_14_0.015 BBL_5_2.0 BBU_5_2.0 MACD_12_26_9".split()
df['FPT'][indicators].sort_index(axis=1).head()

indicators,ADX_14,BBL_5_2.0,BBU_5_2.0,CCI_14_0.015,MACD_12_26_9,RSI_14,SMA_10,close
3010,13.851602,21.497491,22.072813,40.520133,-0.075523,46.346402,21.651595,21.574543
3011,13.520983,21.437136,22.092073,24.071003,-0.078584,46.346402,21.697826,21.574543
3012,12.677717,21.437136,22.092073,69.613235,-0.059599,51.485340,21.733784,21.831383
3013,11.894684,21.473957,21.962790,61.669685,-0.050192,49.939998,21.751762,21.754332
3014,11.521861,21.488789,21.906863,20.613365,-0.042249,49.939998,21.749194,21.754332


In [61]:
tickers = "FPT VNM".split()
indicators = "close ADX_14 SMA_10 RSI_14 CCI_14_0.015 BBL_5_2.0 BBU_5_2.0 MACD_12_26_9".split()
cols = pd.MultiIndex.from_product([tickers, indicators]).to_flat_index().sort_values()

In [62]:
_df = df.copy()
_df.columns = _df.columns.to_flat_index()

In [63]:
_df[cols].head()

,"(FPT, ADX_14)","(FPT, BBL_5_2.0)","(FPT, BBU_5_2.0)","(FPT, CCI_14_0.015)","(FPT, MACD_12_26_9)","(FPT, RSI_14)","(FPT, SMA_10)","(FPT, close)","(VNM, ADX_14)","(VNM, BBL_5_2.0)","(VNM, BBU_5_2.0)","(VNM, CCI_14_0.015)","(VNM, MACD_12_26_9)","(VNM, RSI_14)","(VNM, SMA_10)","(VNM, close)"
3010,13.851602,21.497491,22.072813,40.520133,-0.075523,46.346402,21.651595,21.574543,30.447489,114.797292,117.636256,17.324811,-1.545282,45.936886,114.351438,115.500671
3011,13.520983,21.437136,22.092073,24.071003,-0.078584,46.346402,21.697826,21.574543,30.045572,113.029411,117.740929,-25.297455,-1.634328,41.463615,114.103112,113.190659
3012,12.677717,21.437136,22.092073,69.613235,-0.059599,51.485340,21.733784,21.831383,29.686886,112.035652,117.579683,-20.243286,-1.685468,41.463615,114.039587,113.190659
3013,11.894684,21.473957,21.962790,61.669685,-0.050192,49.939998,21.751762,21.754332,29.589630,112.344153,116.647479,7.374662,-1.554302,46.452741,114.518915,115.096420
3014,11.521861,21.488789,21.906863,20.613365,-0.042249,49.939998,21.749194,21.754332,27.619653,111.929167,117.616869,86.247082,-1.291010,50.703324,115.119518,116.886681


In [ ]:
df.ta.strategy(TechnicalIndicator)
df.dropna(inplace=True)

In [ ]:
indicators = "close SMA_10 RSI_14 CCI_14_0.015 BBL_5_2.0 BBU_5_2.0 MACD_12_26_9".split()
df[indicators].head()

In [ ]:
df.ta.rsi(length=20, append=True)
df.ta.natr(length=20, scalar=1, append=True)
df.ta.log_return(length=5, append=True)
df.ta.log_return(length=20, append=True)
df.ta.percent_return(length=5, append=True)
df.ta.percent_return(length=20, append=True)

# trend setup
# df.ta.sma(50, append=True)
df['close>sma50'] = (df['close'] > df.ta.sma(50)).astype(int)
df['close>sma100'] = (df['close'] > df.ta.sma(100)).astype(int)
df['close>sma200'] = (df['close'] > df.ta.sma(200)).astype(int)

# df.ta.ema(10, append=True)
# df.ta.ema(20, append=True)
df['ema10>ema20'] = (df.ta.ema(10) > df.ta.ema(20)).astype(int)

donchian_20 = ta.donchian(df['high'], df['close'], lower_length=20, upper_length=20)
donchian_50 = ta.donchian(df['high'], df['close'], lower_length=50, upper_length=50)
df['higher_low'] = (donchian_20['DCL_20_20'] > donchian_50['DCL_50_50']).astype(int)
df['breakout'] = (df['close'] > donchian_20['DCU_20_20'].shift(1)).astype(int)

# volume confirm
df['volume_breakout'] = (df['volume'] > ta.sma(df['volume'], 20)).astype(int)

# df.ta.tsignals(ta.ema(df['close'], 10) > ta.ema(df['close'], 20), append=True)
# df.ta.log_return(append=True)
# df.ta.percent_return(append=True)
# df.ta.roc(append=True)
# df.ta.stoch(append=True)
# df.ta.stochrsi(append=True)
# df.ta.tsi(append=True)

# df.describe()
df.dropna(inplace=True)
df.head()
df.to_csv('temp/features.csv', index=False)